In [1]:
# -*- coding: utf-8 -*-
'''
Created on Wed Oct  7 12:14:39 2020

@author: Alexander Crabtree
            acrabtree15@gmail.com
'''
import requests
import json
import pandas as pd
import numpy as np
import math
import sys
import itertools
import threading
import time
from astropy import units as u
from astroquery.irsa_dust import IrsaDust
from astropy.coordinates import Angle,SkyCoord
from astropy.coordinates.name_resolve import NameResolveError
from bs4 import BeautifulSoup


def getLink(name):
    '''
    Grabs the link from this site for the Host name of supernovae in CSV,
    returns the parsed page
    '''
    try:
        link = "http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?"
        inputs = {'objname': name,
				'extend': 'no',
				'hconst': '73.04',
				'omegam': '0.27',
				'omegav': '0.73',
				'corr_z': '1',
				'out_csys': 'Equatorial',
				'out_equinox': 'J2000.0',
				'obj_sort': "RA or Longitude",
				'of': 'pre_text',
				'zv_breaker': '30000.0',
				'list_limit':'10',
				'img_stamp': 'YES'}
        page = requests.get(link, params = inputs)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup
    except requests.ConnectionError:
        soup = ''
        return soup

'''
Takes parsed page from getlink and searches for all velocites listed below
then returns the data.
'''
def scrapeValues(soup):
	
    try:
        soup = soup.find("a", attrs={'name':'DerivedValues_0'})
        velocities = soup.next_sibling.next_sibling.next_sibling.find("pre")
        Helio = list(velocities.children)[2]
        VGS = list(velocities.children)[16]
        Helio = Helio.lstrip('\n')
        VGS = VGS.lstrip('\n')
        Hvals = [int(s) for s in Helio.split() if s.isdigit()]
        VGSVals = [int(s) for s in VGS.split() if s.isdigit()]
        vels = [Hvals[0],Hvals[1],VGSVals[0],VGSVals[1]]
        return vels
    except: 
        vels = [np.nan,np.nan,np.nan,np.nan]
        return vels

def get_coords(gals):
    """
    Takes list of galaxies and looks up their coordinates by name.
    If no name found: warn, skip, remove galaxy from list

    Returns:
        gals: list of galaxies minus those that weren't found
        start_coord: list of coordinates corresponding to center of galaxies in 'gals'
    """
    start_coord = []
    #bar = FillingCirclesBar('Loading galaxies', max = len(gals))
    try:
        tempCoord = SkyCoord.from_name(str(gals), frame = 'icrs')
        start_coord= tempCoord
            #bar.next()
    except NameResolveError:
        #if gals != str(''):
            #print('\nSkipping',gals,'because it couldn\'t be found.')
            #start_coord= ''
        #else:
            start_coord= np.nan
    #bar.finish()
    return(start_coord)


/var/folders/lm/1s7yhzbn56l3gt6dp_gzhtgc0000gn/T/ipykernel_86217/4064121476.py:18: DeprecationWarning: the ``irsa_dust`` module has been moved to astroquery.ipac.irsa.irsa_dust, please update your imports.
  from astroquery.irsa_dust import IrsaDust


In [2]:

def Dist_mod():
    def Dist_mod_empty(hv, hv_err):
        ''' 
        Returns empty nan series if condition is met
        '''
        distance_mod_cor, distance_mod_cor_err = np.nan, np.nan
        return distance_mod_cor, distance_mod_cor_err
    def Distance_mod_cor(hv, hv_err):
        '''
        Takes host_velocity and host_vel_err data and returns distance modulus
        and distance modulus err in a series
        '''
        h0 = 73.04
        h0err = 1.04
	# From Riess et al 2022 SH0ES result

        try:
            distance_mod_cor = 5*math.log(float(hv)/h0,10)+25 #hubble flow
            distance_mod_cor_err = math.sqrt(((5*float(hv_err))/(float(hv)*math.log(10,10)))**2+((5*200)/(float(hv)*math.log(10,10)))**2 + ((5*5.0)/(h0*math.log(10,10)))**2)
        except Exception:
            distance_mod_cor, distance_mod_cor_err = np.nan, np.nan

        return  distance_mod_cor, distance_mod_cor_err

    dist= pd.Series(swift.apply(lambda row: Distance_mod_cor(row['host_velocity'], row['host_vel_err']) if pd.notna(row['host_velocity']) else Dist_mod_empty(row['host_velocity'], row['host_vel_err']), axis=1))
    dist= pd.DataFrame(dist.tolist(), columns=['Hubble_dm', 'Hubble_dm_err'], index=dist.index)
    dist= dist.replace({r'^\s*$'}, np.nan, regex=True)
    return dist


In [3]:
import requests
import json
import pandas as pd
import numpy as np
import math
import sys
import itertools
import threading
import time
from astropy import units as u
from astroquery.irsa_dust import IrsaDust
from astropy.coordinates import Angle,SkyCoord
from astropy.coordinates.name_resolve import NameResolveError
from bs4 import BeautifulSoup


def getLink(name):
    '''
    Grabs the link from this site for the Host name of supernovae in CSV,
    returns the parsed page
    '''
    try:
        link = "http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?"
        inputs = {'objname': name,
				'extend': 'no',
				'hconst': '73.04',
				'omegam': '0.27',
				'omegav': '0.73',
				'corr_z': '1',
				'out_csys': 'Equatorial',
				'out_equinox': 'J2000.0',
				'obj_sort': "RA or Longitude",
				'of': 'pre_text',
				'zv_breaker': '30000.0',
				'list_limit':'10',
				'img_stamp': 'YES'}
        page = requests.get(link, params = inputs)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup
    except requests.ConnectionError:
        soup = ''
        return soup

'''
Takes parsed page from getlink and searches for all velocites listed below
then returns the data.
'''
def scrapeValues(soup):
	
    try:
        soup = soup.find("a", attrs={'name':'DerivedValues_0'})
        velocities = soup.next_sibling.next_sibling.next_sibling.find("pre")
        Helio = list(velocities.children)[2]
        VGS = list(velocities.children)[16]
        Helio = Helio.lstrip('\n')
        VGS = VGS.lstrip('\n')
        Hvals = [int(s) for s in Helio.split() if s.isdigit()]
        VGSVals = [int(s) for s in VGS.split() if s.isdigit()]
        vels = [Hvals[0],Hvals[1],VGSVals[0],VGSVals[1]]
        return vels
    except: 
        vels = [np.nan,np.nan,np.nan,np.nan]
        return vels

def get_coords(gals):
    """
    Takes list of galaxies and looks up their coordinates by name.
    If no name found: warn, skip, remove galaxy from list

    Returns:
        gals: list of galaxies minus those that weren't found
        start_coord: list of coordinates corresponding to center of galaxies in 'gals'
    """
    start_coord = []
    #bar = FillingCirclesBar('Loading galaxies', max = len(gals))
    try:
        tempCoord = SkyCoord.from_name(str(gals), frame = 'icrs')
        start_coord= tempCoord
            #bar.next()
    except NameResolveError:
        #if gals != str(''):
            #print('\nSkipping',gals,'because it couldn\'t be found.')
            #start_coord= ''
        #else:
            start_coord= np.nan
    #bar.finish()
    return(start_coord)

def coord_breakup(coord):
    """
    Breaks up a coordinate into its components
    """
    if pd.isna(coord) == True:
        ra = np.nan
        dec = np.nan
        return ra, dec
    else:
        ra = Angle(coord.ra.hour,unit = u.hour)
        dec = coord.dec
        return ra, dec


def Redshift(soup):
    '''
    Takes parsed page from getlink and searches for redshift then returns that data
    '''
    try:
        soup = soup.find("a", attrs={'name':'BasicData_0'})
        soup = soup.next_sibling.next_sibling.next_sibling.find("pre")
        redshift = list(soup.children)[0]
        # print(redshift)
        redshift = str(redshift).split("Redshift",1)[1]
        # print(redshift)
        redshift = redshift.split(":",1)[1]
        redshift = redshift.split(" +",1)[0].strip(' ')
        return(redshift)
    except:
        redshift = np.nan
        return redshift
   

def Morphology(soup):
    '''
    Takes parsed page from getlink and searches for morphology then returns the data
    '''
    try:
        soup = soup.find("a", attrs={'name':'BasicData_0'})
        morphology = soup.next_sibling.next_sibling.next_sibling.find("pre")
        morphology = list(morphology.children)[2]
        morphology = (morphology.split(": ",4)[4]).rstrip()
        return morphology
    except:
        morphology = np.nan
        return morphology


def LatLong(soup):
    '''
    Takes parsed page from getlink and searches for Latitude and Longitude then returns the data
    '''
    try:
        soup = soup.find("a", attrs={'name':'Positions_0'})
        coords = soup.next_sibling.next_sibling.find("pre")
        coords = list(coords.children)[4]
        coords = (coords.split("Galactic ",1)[1]).lstrip()
        long, lat = str(coords.split()[0]), str(coords.split()[1])
 
        return long, lat
    except:
        long, lat= np.nan,np.nan
        return long, lat  
 

def SN_host_ra_dec():
    def jprint(obj):
        '''
        Used to interpret the data from astrocats catalog
        '''
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)

    '''
    Requests data ra, dec, and host data of all supernovae in astrocats catalog
    '''
    sn_data= requests.get("https://api.astrocats.space/catalog/ra+dec+host?first")
    sn_catalog= sn_data.json()

    def SNHost(SNname):
        ''' 
        Checks if any CSV Supernova names are in astrocats catalog
        If yes returns Host Name data, If no returns nan value
        '''
        
        if SNname in sn_catalog:
            try:
                host_names= sn_catalog[SNname]['host']['value']
            except Exception:
                host_names= np.nan
        else:
            host_names= np.nan

        return host_names

    def SNRa(SNname):
        ''' 
        Checks if any CSV Supernova names are in astrocats catalog
        If yes returns Ra data, If no returns nan value
        '''
        
        if SNname in sn_catalog:
            try:
                snra= "'%s" % sn_catalog[SNname]['ra']['value']
            except Exception: 
                snra= np.nan
        else:
            snra= np.nan
        return snra

    def SNDec(SNname):
        ''' 
        Checks if any CSV Supernova names are in astrocats catalog
        If yes returns Dec data, If no returns nan value
        '''
        
        if SNname in sn_catalog:
            try:
                sndec= sn_catalog[SNname]['dec']['value']
            except Exception: #if no dec value input what is given for dec 
                sndec= np.nan
        else:
            sndec= np.nan
        return sndec
        
    '''
    The next 3 lines uses pandas apply function and lambda function to quickly
    parse supernovae names into the 3 above functions and saves data to
    relavent columns in CSV
    '''

    swift["HostName"]= swift.apply(lambda row: SNHost(row['SNname']), axis=1)
    swift["SNra"]= swift.apply(lambda row: SNRa(row['SNname']), axis=1)
    swift["SNdec"]= swift.apply(lambda row: SNDec(row['SNname']), axis=1)

def GrabSNtypes():
    def SNtype(SNname):
        '''
        Takes Supernovae names from CSV and looks them up in the URL below
        Then parses the page for the supernovae type and returns it
        '''
        
        url = "https://www.wis-tns.org/object/"+SNname

        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            sn_type= soup.find_all('div', class_= 'value')
            sn_type= sn_type[1].get_text()
            if str('---') == sn_type:
                sn_type= np.nan
            else:
                sn_type= sn_type.replace('|'.join(['SN']), '')
        except Exception:
            sn_type= np.nan
        ''' 
        Also collects any Host names not found in SN_host_ra_dec() function
        '''
        try:
            sn_host= soup.find_all('div', class_= 'field field-hostname')
            sn_host= sn_host[0].span.next_sibling.text

        except Exception:
            sn_host= np.nan


        return sn_host, sn_type

    def SNtype_empty(SNname):
            ''' 
            Returns empty nan series if condition is met
            '''
            sn_type, sn_host= np.nan, np.nan
            return sn_host, sn_type


    swift_temp= pd.DataFrame({'SNname':swift['SNname'], 'HostName':swift['HostName'], 'type':swift['type']})
    rep= '|'.join(['SN'])
    swift_temp['SNname']= swift_temp['SNname'].str.replace(rep, '')

    sntype= pd.Series(swift_temp.apply(lambda row: SNtype(row['SNname']) if pd.notna(row['SNname']) and (pd.isna(row['type']) or pd.isna(row['HostName'])) else SNtype_empty(row['SNname']), axis=1))
    sntype= pd.DataFrame(sntype.tolist(), columns=['HostName', 'type'], index=sntype.index)
    sntype= sntype.replace({r'^\s*$'}, np.nan, regex=True)
    return sntype


def getAVbest(inputcoordinates):
    '''
    Coordinates are input as a single string. Output is the recommended Av value for MW reddening, error, and reference
    '''
    
    #inputcoordinates = sys.argv[1]
    testCoords = SkyCoord(inputcoordinates,frame='fk5')
    #print('\n-----\nReading input files...')
    inFile = 'Brown_Walker_table_1.dat'
    inTable = pd.read_csv(inFile,header=None,delimiter=' ')
    ra = Angle(inTable.iloc[:,1])
    dec = Angle(inTable.iloc[:,2])
    sourceCoords = SkyCoord(ra,dec,frame='fk5')
    
    #print('Calculating separation from table coordinates')
    separations = testCoords.separation(sourceCoords).arcminute
    # compare to the distances in the table
    within = np.less(separations,inTable.iloc[:,3])
    
    # Are any of the input coordinates within the tabulated distance 
    # of the coordinates in the table?
    correctedAV = np.where(within,inTable.iloc[:,4],None) #get calculated value
    fix=any(within)
    #print('fix?',fix)
    
    if fix:
        AV = next((item for item in correctedAV if item is not None),None)
        correctedAVerr = np.where(within,inTable.iloc[:,5],None) #get calculated val
        newAVerr = next((item for item in correctedAVerr if item is not None),None)
        AVerr = math.sqrt((int(float(newAVerr)))**2+(int(AV)*0.1)**2)
        sources=np.where(within,inTable.iloc[:,6],None)
        source = next((item for item in sources if item is not None),None)+",S_F_2011"
        
    if not fix:
        AVtable = IrsaDust.get_extinction_table(testCoords,show_progress = False)
        AV=AVtable['A_SandF'][2]
        AVerr = AV*0.1
        source = 'S_F_2011'

    #print(AV, AVerr, source)
    return (AV, AVerr, source)


def AV_best():
    def AV_empty(Ra, Dec):
        ''' 
        Returns empty nan values if condition is met
        '''
        AV, AVerr, AVsour= np.nan, np.nan, np.nan
        return AV, AVerr, AVsour
    def GrabAVbest(Ra,Dec):
        '''
        Takes Supernovae Ra,Dec data and puts in in the right format before
        runing it through the getAVbest function, if there is an Exception
        returns nan values
        '''
        ra_fix, dec_fix = Ra, Dec
        ra_fix= ra_fix.replace("'", "")
        combined= SkyCoord(ra=ra_fix, dec=dec_fix, unit=(u.hour, u.deg)).to_string('hmsdms')
        try:
            AV, AVerr, AVsour= getAVbest(combined)
        except Exception:
            AV, AVerr, AVsour= np.nan, np.nan, np.nan
        
        return AV, AVerr, AVsour

    avdata= pd.Series(swift.apply(lambda row: GrabAVbest(row['SNra'], row['SNdec']) if (pd.isna(row['AV']) and pd.notna(row['SNra'])) else AV_empty(row['SNra'], row['SNdec']), axis= 1))
    avdata= pd.DataFrame(avdata.to_list(), columns=['AV', 'AVerr', 'AVsour'], index=avdata.index)
    avdata= avdata.replace({r'^\s*$'}, np.nan, regex=True)

    return avdata



In [4]:
def AllHostData():
    '''
    Collects all data on Host names by running appropriate data through related 
    function or empty function.
    '''
    def host_coords(HostName):
        ra,dec= coord_breakup(get_coords(HostName))
        return ra,dec
    def host_coords_empty(HostName):
        ra,dec= np.nan, np.nan
        return ra, dec
    
    def host_lat_long(HostName):
        gal_link=getLink(HostName)
        lon,lat= LatLong(gal_link)
        return lon, lat
    def host_lat_long_empty(HostName):
        lon,lat= np.nan, np.nan
        return lon, lat
    
    def redshift(HostName):
        gal_link=getLink(HostName)
        red= Redshift(gal_link)
        return red
    def redshift_empty(HostName):
        red= np.nan
        return red
    
    def morphology(HostName):
        gal_link=getLink(HostName)
        morph= Morphology(gal_link)
        return morph
    def morphology_empty(HostName):
        morph= np.nan
        return morph
    
    def velocities(HostName):
        gal_link=getLink(HostName)
        vels= scrapeValues(gal_link)
        return vels
    def velocities_empty(HostName):
        vels= np.nan, np.nan, np.nan, np.nan
        return vels
    
    '''
    All 5 lines uses pandas apply and the lambda function with an if else staments inside. Checks if HostName cell is not empty
    and checks specfic cell is empty for all cases. If so runs them through their corresponding functions if not runs them
    through their corresponding empty functions
    '''

    coord= pd.Series(swift.apply(lambda row: host_coords(row['HostName']) if (pd.notna(row['HostName']) and pd.isna(row['HostRa'])) else host_coords_empty(row['HostName']), axis=1))
    coord= pd.DataFrame(coord.tolist(), columns=['HostRa','HostDec'], index=coord.index)

    lon_lat= pd.Series(swift.apply(lambda row: host_lat_long(row['HostName']) if (pd.notna(row['HostName']) and pd.isna(row['Long'])) else host_lat_long_empty(row['HostName']), axis=1))
    lon_lat= pd.DataFrame(lon_lat.tolist(), columns=['Long','Lat'], index=lon_lat.index)
    lon_lat= lon_lat.replace({r'^\s*$'}, np.nan, regex=True)

    reds= pd.Series(swift.apply(lambda row: redshift(row['HostName']) if (pd.notna(row['HostName']) and pd.isna(row['Redshift'])) else redshift_empty(row['HostName']), axis=1))
    reds= pd.DataFrame(reds.tolist(), columns=['Redshift'], index=reds.index)
    reds= reds.replace({r'^\s*$'}, np.nan, regex=True)

    morp= pd.Series(swift.apply(lambda row: morphology(row['HostName']) if (pd.notna(row['HostName']) and pd.isna(row['Morphology'])) else morphology_empty(row['HostName']), axis=1))
    morp= pd.DataFrame(morp.tolist(), columns=['Morphology'], index=morp.index)
    morp= morp.replace({r'^\s*$'}, np.nan, regex=True)

    hvels= pd.Series(swift.apply(lambda row: velocities(row['HostName']) if (pd.notna(row['HostName']) and pd.isna(row['host_velocity'])) else velocities_empty(row['HostName']), axis=1))
    hvels= pd.DataFrame(hvels.tolist(), columns=['host_velocity','host_vel_err', 'host_vel_corr', 'host_vel_corr_err'], index=hvels.index)

    all_data= pd.concat([coord, lon_lat, reds, morp, hvels], axis=1)
    
    return all_data


In [5]:
def coord_breakup(coord):
    """
    Breaks up a coordinate into its components
    """
    if pd.isna(coord) == True:
        ra = np.nan
        dec = np.nan
        return ra, dec
    else:
        ra = Angle(coord.ra.hour,unit = u.hour)
        dec = coord.dec
        return ra, dec


In [6]:

def Best_Distances():
    def Distances_empty(host_vel, Hubble_dm, Hubble_dm_err, Hubble_dm_ref, SBF_dm, SBF_dm_err, SBF_ref, Cep_dm, Cep_dm_err, Cep_ref, TRGB_dm, TRGB_dm_err, TRGB_dm_ref):
        '''
        returns nans to series if there is no host_velocity value available
        '''
        best_dm, best_dm_err, best_dm_meth, best_dm_ref= np.nan, np.nan, np.nan, np.nan
        return best_dm, best_dm_err, best_dm_meth, best_dm_ref

    def Distances(host_vel, Hubble_dm, Hubble_dm_err, Hubble_dm_ref, SBF_dm, SBF_dm_err, SBF_ref, Cep_dm, Cep_dm_err, Cep_ref, TRGB_dm, TRGB_dm_err, TRGB_dm_ref):
        '''
        If there is a host_velocity value, checks if there are values for sbf_dist and cep_dist.
        If none for both returns dist_mod as Hubble flow distance.
        If SBF, returns sbf_dist as SBF.
        If Cepheid, returns cep_dist as Cepheid.
        '''
        try:
            if pd.isnull(Cep_dm)==True and pd.isnull(SBF_dm)==True:
                best_dm= Hubble_dm
                best_dm_err= Hubble_dm_err
                best_dm_meth= "HF"
                best_dm_ref= Hubble_dm_ref
            elif pd.isnull(Cep_dm)==True and pd.isnull(SBF_dm)==False:
                best_dm= SBF_dm
                best_dm_err= SBF_dm_err
                best_dm_method= "SBF"
                best_dm_ref= SBF_dm_ref
            else:
                best_dm= Cep_dm
                best_dm_err= Cep_dm_err
                best_dm_meth= "Cepheid"
                best_dm_ref= Cep_dm_ref
        
        except Exception:
                    best_dm, best_dm_err, best_dm_meth, best_dm_ref= np.nan, np.nan, np.nan, np.nan

        return best_dm, best_dm_err, best_dm_meth, best_dm_ref
        
    best_dist= pd.Series(swift.apply(lambda row: Distances(row['host_velocity'], row['Hubble_dm'], row['Hubble_dm_err'], row['Hubble_dm_ref'], row['SBF_dm'], row['SBF_dm_err'], row['SBF_dm_ref'], row['Cep_dm'], row['Cep_dm_err'], row['Cep_dm_ref'],row['TRGB_dm'], row['TRGB_dm_err'], row['TRGB_dm_ref']) if pd.notna(row['host_velocity']) else Distances_empty(row['host_velocity'], row['Hubble_dm'], row['Hubble_dm_err'], row['Hubble_dm_ref'], row['SBF_dm'], row['SBF_dm_err'], row['SBF_dm_ref'], row['Cep_dm'], row['Cep_dm_err'], row['Cep_dm_ref'],row['TRGB_dm'], row['TRGB_dm_err'], row['TRGB_dm_ref']), axis=1))
    best_dist= pd.DataFrame(best_dist.to_list(), columns=['best_dm', 'best_dm_err', 'best_dm_method', 'best_dm_ref'], index= best_dist.index)
    best_dist= best_dist.replace({r'^\s*$'},np.nan, regex=True)
    print(best_dist)
    return best_dist


In [7]:
global swift
inputcsv='test.csv'
swift= pd.read_csv(inputcsv,on_bad_lines='warn',delimiter=',')
#print(swift)
#print(swift['HostName'])
swift= swift.replace({r'anon',r'Anon',r'AnonHost', r'^\s*$'},np.nan, regex=True)
swift= swift.fillna(AllHostData())

print(swift['best_dm'])
#swift['best_dm']=swift['Cep_dm'] if pd.notna(swift['Cep_dm']) else swift['SBF_dm'] if pd.notna(swift['SBF_dm']) else swift['Hubble_dm'] 



#df.animal = df.animal + ", " + df.type + ", " + \
#    df.age.astype(str) + " year" + np.where(df.age > 1, 's', '')


#df.best_dm= df.Cep_dm

swift['best_dm']=swift['Hubble_dm']
#swift.loc[ np.notna(swift['Cep_dm']),'best_dm'] = swift['Cep_dm']

swift['best_dm']=swift['Hubble_dm']



#df['FirstName']=df['ID'].apply(lambda x: 'Matt' if x==103 else '')


#swift_dist= Best_Distances()
#print(swift_dist)
#swift= swift.drop(swift.columns.intersection(swift_dist.columns), axis=1).join(swift_dist)

#swift.loc[pd.notna(swift['Cep_dm']), 'Best_dm'] = swift['Cep_dm']

#swift=swift.fillna('')
swift.to_csv('New'+inputcsv, index= False)
print(swift['best_dm'])

 

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9    33.18480636
Name: best_dm, dtype: object
0    12
1    12
2    12
3    12
4    12
5    12
6    12
7    12
8    12
9    12
Name: best_dm, dtype: int64


In [8]:
pd.notna(swift["Cep_dm"])



0     True
1     True
2     True
3     True
4    False
5    False
6    False
7    False
8    False
9    False
Name: Cep_dm, dtype: bool

In [9]:
swift.Cep_dm[pd.notna(swift["Cep_dm"])]

0    13
1    13
2    13
3    13
Name: Cep_dm, dtype: object

In [10]:
swift.best_dm[pd.notna(swift["Cep_dm"])]

0    12
1    12
2    12
3    12
Name: best_dm, dtype: int64

In [11]:
swift.Cep_dm[pd.notna(swift["Cep_dm"])]


0    13
1    13
2    13
3    13
Name: Cep_dm, dtype: object

In [12]:
swift.best_dm=swift.Hubble_dm
swift.loc[(pd.notna(swift["TRGB_dm"]),"best_dm")]=swift.loc[(pd.notna(swift["TRGB_dm"]),"TRGB_dm")]
swift.loc[(pd.notna(swift["SBF_dm"]),"best_dm")]=swift.loc[(pd.notna(swift["SBF_dm"]),"SBF_dm")]

swift.loc[(pd.notna(swift["Cep_dm"]),"best_dm")]=swift.loc[(pd.notna(swift["Cep_dm"]),"Cep_dm")]



In [13]:
swift.best_dm


0    13
1    13
2    13
3    13
4    14
5    14
6    15
7    15
8    12
9    12
Name: best_dm, dtype: object